In [1]:
from tensorflow.python.keras.layers import Input,Dense,Reshape,Lambda,RNN,Concatenate,StackedRNNCells
from tensorflow.python.keras.models import Model,load_model
from tensorflow.python.keras.utils import multi_gpu_model
from tensorflow.python.keras.optimizers import Adam,SGD
from tensorflow.python.keras.callbacks import LearningRateScheduler,Callback,ModelCheckpoint,TensorBoard

import tensorflow as tf
import datetime
import os
import numpy as np
import pickle

%env PYTHONHASHSEED=0
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
os.environ["CUDA_VISIBLE_DEVICES"]="1"

/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/

env: PYTHONHASHSEED=0


/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site

In [1]:
from sklearn.model_selection import train_test_split
#import tensorflow_datasets as tfds
#features
n2v_start = np.load("start_emb.npy")
n2v_end = np.load("end_emb.npy")
lanes = np.load("lanes.npy")
oneway = np.load("oneway.npy")
roadtype = np.load("highway.npy")
maxspeed = np.load("maxspeed.npy")
nodetype = np.load("node_highway.npy")
#target
meanspeed = np.load("meanspeed.npy")

#split
#n2v_start_train, n2v_start_test, n2v_end_train, n2v_end_test, lanes_train, lanes_test, oneway_train, oneway_test, roadtype_train, roadtype_test, maxspeed_train, maxspeed_test, nodetype_train, nodetype_test, meanspeed_train, meanspeed_test = train_test_split(n2v_start, n2v_end, lanes,oneway,roadtype,maxspeed,nodetype,meanspeed)

n2v_start_train, n2v_start_test = tfds.load(n2v_start, split=['train', 'test'])
n2v_end_train, n2v_end_test = tfds.load(n2v_end, split=['train', 'test'])
lanes_train, lanes_test = tfds.load(lanes, split=['train', 'test'])
oneway_train, oneway_test = tfds.load(oneway, split=['train', 'test'])
roadtype_train, roadtype_test = tfds.load(roadtype, split=['train', 'test'])
maxspeed_train, maxspeed_test = tfds.load(maxspeed, split=['train', 'test'])
nodetype_train, nodetype_test = tfds.load(nodetype, split=['train', 'test'])
meanspeed_train, meanspeed_test = tfds.load(meanspeed, split=['train', 'test'])

del n2v_start
del n2v_end
del lanes
del oneway
del roadtype
del maxspeed
del nodetype
del meanspeed

ModuleNotFoundError: No module named 'sklearn'

In [3]:
n2v_start_train=np.load('/home/fyp2/Ideaer/Madrid/split/n2v_start_train.npy')
n2v_start_test=np.load('/home/fyp2/Ideaer/Madrid/split/n2v_start_test.npy')

n2v_end_train=np.load('/home/fyp2/Ideaer/Madrid/split/n2v_end_train.npy')
n2v_end_test=np.load('/home/fyp2/Ideaer/Madrid/split/n2v_end_test.npy')

lanes_train=np.load('/home/fyp2/Ideaer/Madrid/split/lanes_train.npy')
lanes_test=np.load('/home/fyp2/Ideaer/Madrid/split/lanes_test.npy')

oneway_train=np.load('/home/fyp2/Ideaer/Madrid/split/oneway_train.npy')
oneway_test=np.load('/home/fyp2/Ideaer/Madrid/split/oneway_test.npy')

roadtype_train=np.load('/home/fyp2/Ideaer/Madrid/split/roadtype_train.npy')
roadtype_test=np.load('/home/fyp2/Ideaer/Madrid/split/roadtype_test.npy')

maxspeed_train=np.load('/home/fyp2/Ideaer/Madrid/split/maxspeed_train.npy')
maxspeed_test=np.load('/home/fyp2/Ideaer/Madrid/split/maxspeed_test.npy')

nodetype_train=np.load('/home/fyp2/Ideaer/Madrid/split/nodetype_train.npy')
nodetype_test=np.load('/home/fyp2/Ideaer/Madrid/split/nodetype_test.npy')

meanspeed_train=np.load('/home/fyp2/Ideaer/Madrid/split/meanspeed_train.npy')
meanspeed_test=np.load('/home/fyp2/Ideaer/Madrid/split/meanspeed_test.npy')

In [4]:
embinplist = list()
inplist = list()

n2v_start_inp = Input((20,))
n2v_start_reg = Dense(20,activation = "relu", use_bias = True)(n2v_start_inp)
inplist.append(n2v_start_inp)
embinplist.append(n2v_start_reg)

n2v_end_inp = Input((20,))
n2v_end_reg = Dense(20,activation = "relu", use_bias = True)(n2v_end_inp)
inplist.append(n2v_end_inp)
embinplist.append(n2v_end_reg)

lanes_inp = Input((1,))
lanes_reg= Dense(1,activation = "relu", use_bias = True)(lanes_inp)
inplist.append(lanes_inp)
embinplist.append(lanes_reg)

oneway_inp = Input((2,))
oneway_reg= Dense(2,activation = "relu", use_bias = True)(oneway_inp)
inplist.append(oneway_inp)
embinplist.append(oneway_reg)

roadtype_inp = Input((13,))
roadtype_reg= Dense(8,activation = "relu", use_bias = True)(roadtype_inp)
inplist.append(roadtype_inp)
embinplist.append(roadtype_reg)

maxspeed_inp = Input((1,))
maxspeed_reg= Dense(1,activation = "relu", use_bias = True)(maxspeed_inp)
inplist.append(maxspeed_inp)
embinplist.append(maxspeed_reg)

nodetype_inp = Input((11,))
nodetype_reg= Dense(8,activation = "relu", use_bias = True)(nodetype_inp)
inplist.append(nodetype_inp)
embinplist.append(nodetype_reg)

combine = Concatenate()(embinplist)
proj = Dense(32,activation = "relu",use_bias = True)(combine)
proj = Dense(1,activation = "linear")(proj)

emb = Model(inplist,proj)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
emb.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 2)]          0                                            
______________________________________________________________________________________________

In [6]:
adam=Adam(lr=0.001,beta_1=0.9,beta_2=0.999,amsgrad=True)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
emb.compile(optimizer=adam, loss='mae',metrics=['mae'])

In [ ]:
emb.fit([n2v_start_train,n2v_end_train,lanes_train,oneway_train,roadtype_train,maxspeed_train,nodetype_train],
        meanspeed_train,
        epochs=100,batch_size=512,
        validation_data=([n2v_start_test,n2v_end_test,lanes_test,oneway_test,roadtype_test,maxspeed_test,nodetype_test],
        meanspeed_test)
       )

Train on 86992 samples, validate on 28998 samples
